In [1]:
import numpy as np
from scipy import special as sp
import csv
import copy
from constitutive import *

#need matlab to check
import matlab
import matlab.engine


In [2]:
eng = matlab.engine.start_matlab()

In [179]:
eng.run('../domeconduit/Code/RunModelScripts/AddPaths.m', nargout= 0)
eng.addpath('../domeconduit/Code/RunModelScripts/', nargout= 0)
eng.addpath ('../MassMatrix/', nargout= 0 )

In [73]:
# Model parameters
m = {}


# temperature
Tvec = np.array((950, 900, 850, 800, 750))
m['Ti'] = 3
m['T'] = np.float(Tvec[m['Ti']-1] + 273.15)  # temperature [K]

# geometry, all in meters
m['R'] = 50
m['conduit_length'] = 4e3

m['g'] = 9.81    # gravitational acceleration, m/s2


# densities
m['rho_s']  = 2600.
m['rho_l']  = 2200.
m['rho_hd'] = 2200.
m['rho_cd'] = 2200.

# gas constants
m['Rw'] = 461.5         # J/(kg K)
m['Rc'] = 188.9         # J/(kg K)
m['B']  = 18.02/44.01;  # molar mass ratio water/co2
m['eta_g'] = 1e-4       # gas viscosity, Pa s

# permeability 
m['kc']     = 1e-13    # magma permeability constant, m2
m['phi_gc'] = 0.0     # percolation threshold

# wall rock permeability properties
klw        = {}       
klw['top'] = 1e-14    # wall rock permeability at top, m2
klw['mi']  = 3.2      # wall rock permeability exponent in Manning & Ingebritsen 1999
klw['L']   = m['R']   # length scale for wall rock gas percolation
m['klw']   = klw

# hydrostatic pressure
m['phydro'] = {'slope':9810}

# friction
fr = {}
fr['v_ref'] = 1e-5    # reference vel for friction, m/s
fr['f0']    = 0.1     # nominal friction coeff
fr['a']     = 0.01    # velocity dependence of friction
fr['A']     = 2*fr['v_ref']*np.exp(-fr['f0']/fr['a'])   # prefactor in MBE
m['fr']     = fr

# normal stress
m['sig']      = {'slope':16677., 'offset':1000000.}




# initialise solid fraction curves
fname = 'chiscurves/chis_ti' + str(m['Ti']) + '.csv'
with open(fname, newline='') as f:
    reader = np.array(list(csv.reader(f)),dtype=np.float)

pp = {}
pp['breaks'] = np.array(reader[:,0]).reshape(len(reader),1)
pp['coefs'] = reader[:-1,1:]
m['pf'] = pp


print(m['fr'])

{'v_ref': 1e-05, 'f0': 0.1, 'a': 0.01, 'A': 9.079985952496972e-10}


### the next few cells are just to check that the python code for constitutive relations is the same as the matlab functions

In [5]:
# check phase fraction derivatives
p = mat_to_py(np.array([100e6,75e6,50e6,25e6]))
phi_g = mat_to_py(np.array([0.1,0.2,0.3,0.3]))
mh = mat_to_py(np.array([0.9,0.92,0.95,0.98]))

h2o, co2, c, phi, rho = density (p, phi_g, mh, m)

pyout = {}
pyout['h2o_d'] = h2o['dissolved']
pyout['co2_d'] = co2['dissolved']
pyout['c1']    = c['c1']
pyout['c2']    = c['c2']
pyout['chi_s'] = phi['chi_s']['chi_s']
pyout['phi_s'] = phi['s']
pyout['phi_l'] = phi['l']
pyout['rho']   = rho['mix']
pyout['dh2o_d']= {'p':h2o['dp']         , 'mh':h2o['dmh']}
pyout['dco2_d']= {'p':co2['dp']         , 'mh':co2['dmh']}
pyout['dc1']   = {'p':c['dc1']['dp']    , 'mh':c['dc1']['dmh']}
pyout['dc2']   = {'p':c['dc2']['dp']    , 'mh':c['dc2']['dmh']}
pyout['dphis'] = {'p':phi['dphis']['dp'], 'mh':phi['dphis']['dmh'], 'phi_g':phi['dphis']['dphi_g']}
pyout['dphil'] = {'p':phi['dphil']['dp'], 'mh':phi['dphil']['dmh'], 'phi_g':phi['dphil']['dphi_g']}


In [6]:

pm = matlab.double(p.tolist())
phigm = matlab.double(phi_g.tolist())
mhm = matlab.double(mh.tolist())

mtmp = copy.deepcopy(m)

mtmp['pf']['breaks'] = matlab.double(mtmp['pf']['breaks'].tolist())
mtmp['pf']['coefs'] = matlab.double(mtmp['pf']['coefs'].tolist())

mtmp.pop('fr', None)

Chi_hd, Chi_cd, c1, c2, chi_s, phi_s, phi_l, rho, dChihd, dChicd, dc1, dc2, dphis, dphil = eng.phasefracderivs(pm, phigm, mhm, mtmp, nargout=14)

mlout = {}
mlout['h2o_d'] = np.array(Chi_hd)
mlout['co2_d'] = np.array(Chi_cd)
mlout['c1']    = np.array(c1)
mlout['c2']    = np.array(c2)
mlout['chi_s'] = np.array(chi_s)
mlout['phi_s'] = np.array(phi_s)
mlout['phi_l'] = np.array(phi_l)
mlout['rho']   = np.array(rho)
mlout['dh2o_d']= np.array(dChihd)
mlout['dco2_d']= np.array(dChicd)
mlout['dc1']   = np.array(dc1)
mlout['dc2']   = np.array(dc2)
mlout['dphis'] = np.array(dphis)
mlout['dphil'] = np.array(dphil)


In [7]:
# prints out and compares all the variables between the matlab and python codes


print('python code: rho = ', pyout['rho'].T)
print('matlab code: rho = ', mlout['rho'].T, '\n')

print('python code: c1 = ', pyout['c1'].T)
print('matlab code: c1 = ', mlout['c1'].T, '\n')

print('python code: c2 = ', pyout['c2'].T)
print('matlab code: c2 = ', mlout['c2'].T, '\n')

print('python code: dh2o_d/dp = ', pyout['dh2o_d']['p'].T)
print('matlab code: dh2o_d/dp = ', mlout['dh2o_d'][:,0].T, '\n')

print('python code: dh2o_d/dmh = ', pyout['dh2o_d']['mh'].T)
print('matlab code: dh2o_d/dmh = ', mlout['dh2o_d'][:,1].T, '\n')

print('python code: dco2_d/dp = ', pyout['dco2_d']['p'].T)
print('matlab code: dco2_d/dp = ', mlout['dco2_d'][:,0].T, '\n')

print('python code: dco2_d/dmh = ', pyout['dco2_d']['mh'].T)
print('matlab code: dco2_d/dmh = ', mlout['dco2_d'][:,1].T, '\n')


print('python code: dc1/dp = ', pyout['dc1']['p'].T)
print('matlab code: dc1/dp = ', mlout['dc1'][:,0].T, '\n')

print('python code: dc1/dmh = ', pyout['dc1']['mh'].T)
print('matlab code: dc1/dmh = ', mlout['dc1'][:,1].T, '\n')


print('python code: dc2/dp = ', pyout['dc2']['p'].T)
print('matlab code: dc2/dp = ', mlout['dc2'][:,0].T, '\n')

print('python code: dc2/dmh = ', pyout['dc2']['mh'].T)
print('matlab code: dc2/dmh = ', mlout['dc2'][:,1].T, '\n')



print('python code: dphis/dp = ', pyout['dphis']['p'].T)
print('matlab code: dphis/dp = ', mlout['dphis'][:,0].T, '\n')

print('python code: dphis/dphig = ', pyout['dphis']['phi_g'].T)
print('matlab code: dphis/dphig = ', mlout['dphis'][:,1].T, '\n')

print('python code: dphis/dmh = ', pyout['dphis']['mh'].T)
print('matlab code: dphis/dmh = ', mlout['dphis'][:,2].T, '\n')


print('python code: dphil/dp = ', pyout['dphil']['p'].T)
print('matlab code: dphil/dp = ', mlout['dphil'][:,0].T, '\n')

print('python code: dphil/dphig = ', pyout['dphil']['phi_g'].T)
print('matlab code: dphil/dphig = ', mlout['dphil'][:,1].T, '\n')

print('python code: dphil/dmh = ', pyout['dphil']['mh'].T)
print('matlab code: dphil/dmh = ', mlout['dphil'][:,2].T, '\n')

python code: rho =  [[2159.0937166  1944.16366888 1722.2065867  1834.24726395]]
matlab code: rho =  [[2159.0937166  1944.16366888 1722.2065867  1834.24726395]] 

python code: c1 =  [[1.03796513 1.03247169 1.02613116 1.01792041]]
matlab code: c1 =  [[1.03796513 1.03247169 1.02613116 1.01792041]] 

python code: c2 =  [[0.9634235  0.96854956 0.97453429 0.98239507]]
matlab code: c2 =  [[0.9634235  0.96854956 0.97453429 0.98239507]] 

python code: dh2o_d/dp =  [[2.08740937e-10 2.38465594e-10 2.87041498e-10 3.88404630e-10]]
matlab code: dh2o_d/dp =  [2.08740937e-10 2.38465594e-10 2.87041498e-10 3.88404630e-10] 

python code: dh2o_d/dmh =  [[0.02569807 0.02094031 0.01584116 0.01013028]]
matlab code: dh2o_d/dmh =  [0.02569807 0.02094031 0.01584116 0.01013028] 

python code: dco2_d/dp =  [[6.31131807e-13 4.99306140e-13 3.12690527e-13 1.25823972e-13]]
matlab code: dco2_d/dp =  [6.31131807e-13 4.99306140e-13 3.12690527e-13 1.25823972e-13] 

python code: dco2_d/dmh =  [[-0.00062137 -0.00046553 -0.

In [8]:
# check viscosity

h2o_d = 5.
phi_s_eta = 0.5
gdot = 1.0e-5

eta1  = viscosity(h2o_d, phi_s_eta, gdot, m)
eta2, eta_m2, eta_phi2 = eng.tdcFV('calc_eta', h2o_d*0.01, phi_s_eta, m['T'], 2, gdot, nargout=3)

print('python code: ', eta1['mix'], eta1['melt'], eta1['xtal'])
print('matlab code: ', eta2, eta_m2, eta_phi2)



python code:  233815.67501426488 5832.851929387548 40.08599529781234
matlab code:  233815.67501426488 5832.851929387548 40.08599529781234


In [9]:
# check solid mass fraction

p = mat_to_py(np.array([100e6,50e6]))
chi_s = solid_mass_frac(p*1e-6, m['pf'])
chi_s['dchisdp'] = solid_mass_frac_deriv(chi_s)


print('python code: chi_s = ', chi_s['chi_s'].T)
print('matlab code: chi_s = ', 0.477586596754590,0.581054531506694)

print('python code: dchis/dp = 1e-8 x', 1e8*chi_s['dchisdp'].T)
print('matlab code: dchis/dp = 1e-8 x', -0.148494868567476,  -0.468613491456686)


python code: chi_s =  [[0.4775866  0.58105453]]
matlab code: chi_s =  0.47758659675459 0.581054531506694
python code: dchis/dp = 1e-8 x [[-0.14849487 -0.46861349]]
matlab code: dchis/dp = 1e-8 x -0.148494868567476 -0.468613491456686


In [10]:
# check permeability
degas =  1
p     =  1e7*mat_to_py(np.array([9.126853500379374,   7.746970469494080,   6.427370851459487,   5.174760554871329]))
phi_g =      mat_to_py(np.array([0.104233779536040,   0.144496818364394,   0.189868604799967,   0.238369127138891]))
dpdz  = -1e4*mat_to_py(np.array([2.110369216779560,   2.026756139387187,   1.930246732696368,   1.828099332132594]))
z     = -1e3*mat_to_py(np.array([3.340000000000000,   2.673333333333333,   2.006666666666667,   1.340000000000000]))

k = permeability(phi_g, degas, z, m)
gvel = gasvels(p, phi_g, dpdz, degas, z, m)

print('python code: kvert = 1e-14 x', k['vert'].T*1e14)
print('matlab code: kvert = 1e-14 x [ 0.011324667428581   0.030169968293476   0.068447797384461   0.135440958492205]\n')

print('python code: klat = 1e-14 x', k['lat'].T*1e14)
print('matlab code: klat = 1e-14 x [ 0.014735594318271   0.035458845924748   0.083690262157321   0.201319910330897]\n')

print('python code: (vg-v) = 1e-7 x', gvel['vg']['vel'].T*1e7)
print('matlab code: (vg-v) = 1e-7 x [0.238992295315426   0.611471684639190   1.321211372616185   2.4759952576299889]\n')

print('python code: ug = 1e-6 x', gvel['ug'].T*1e6)
print('matlab code: ug = 1e-6 x [0.862078463762972   1.817063904698754   3.731607228737932   7.771392559641819]\n')

python code: kvert = 1e-14 x [[0.01132467 0.03016997 0.0684478  0.13544096]]
matlab code: kvert = 1e-14 x [ 0.011324667428581   0.030169968293476   0.068447797384461   0.135440958492205]

python code: klat = 1e-14 x [[0.01473559 0.03545885 0.08369026 0.20131991]]
matlab code: klat = 1e-14 x [ 0.014735594318271   0.035458845924748   0.083690262157321   0.201319910330897]

python code: (vg-v) = 1e-7 x [[0.2389923  0.61147168 1.32121137 2.47599526]]
matlab code: (vg-v) = 1e-7 x [0.238992295315426   0.611471684639190   1.321211372616185   2.4759952576299889]

python code: ug = 1e-6 x [[0.86207846 1.8170639  3.73160723 7.77139256]]
matlab code: ug = 1e-6 x [0.862078463762972   1.817063904698754   3.731607228737932   7.771392559641819]



### now assemble equations for steady-state system

In [74]:
def mbe_vals (z, v, dpdz, h2o_d, phi, rho, m):
    
    phi_s_eta = phi['s']/(1 - var['phi_g'])
    sig_c = m['sig']['slope']*(-z) + m['sig']['offset']
    gdot  = 2*np.absolute(v)/m['R']
    eta   = viscosity(h2o_d, phi_s_eta, gdot, m)
    tau_R = -0.5*m['R']*(dpdz + rho*m['g'])
    
    vfr = m['fr']['A']*np.sinh(tau_R/m['fr']['a']/sig_c)
    
    mbe = {'vfr': vfr, 'eta':eta['mix']}
    return mbe

In [210]:
def ssc (z, var, grad, m):
    """
    order of variables: p, v, phi_g, mh
    order of equations (to be consistent with variable order and have diagonals populated)
        1) solids and liquids continuity
        2) mbe
        3) h2o continuity
        4) co2 continuity
    """
    
    Nz = len(z)
    J  = np.zeros((Nz,4,8))
    Q  = np.zeros((Nz,4))
    
    # necessary constitutive relations
    h2o, co2, c, phi, rho = density (var['p'], var['phi_g'], var['mh'], m)
    gamma = exsolvedco2h2o(var['mh'],m)
    degas = 1.
    gvel = gasvels(var['p'], var['phi_g'], grad['p']['z'], degas, z, m)

    # mbe
    mbe = mbe_vals(z, var['v'], grad['p']['z'], h2o['dissolved'], phi, rho['mix'], m)
    Qmbe = np.power(m['R'],2)/8/mbe['eta']*rho['mix']*m['g']
    Q[:, m['vi']['v']] = Qmbe.reshape(Nz)
    
    J[:, m['vi']['v'], m['vi']['v']]  = 1.  # for v
    J[:, m['vi']['v'], m['vi']['pz']] = np.power(m['R'],2)/8/mbe['eta'].reshape(Nz) # for dpdz
    
    
    
    # continuity
    
    # first reshape some things for simple code
    vrshp  = var['v'].reshape(Nz)
    vgrshp = gvel['vg']['vel'].reshape(Nz)
    
    
    # solids and liquids
    fsl         = m['rho_s']*phi['s']               + m['rho_l']*phi['l']
    dMsldp      = m['rho_s']*phi['dphis']['dp']     + m['rho_l']*phi['dphil']['dp']
    dMsldphig   = m['rho_s']*phi['dphis']['dphi_g'] + m['rho_l']*phi['dphil']['dphi_g']
    dMsldmh     = m['rho_s']*phi['dphis']['dmh']    + m['rho_l']*phi['dphil']['dmh']

    J[:, m['vi']['p'], m['vi']['vz']]     = fsl.reshape(Nz)
    J[:, m['vi']['p'], m['vi']['pz']]     = vrshp*   dMsldp.reshape(Nz)
    J[:, m['vi']['p'], m['vi']['phi_gz']] = vrshp*dMsldphig.reshape(Nz)
    J[:, m['vi']['p'], m['vi']['mhz']]    = vrshp*  dMsldmh.reshape(Nz)
    
    

    # water, separate into liquid and gas parts for slightly more intelligible code
    fhl = h2o['dissolved']*m['rho_l']*phi['l']*c['c1']
    fhg = 1/(1+gamma['g'])*rho['g']*var['phi_g']
    fh  = fhl + fhg
    
    # liquid part
    dMhldp    = m['rho_l']*( h2o['dissolved']*phi['dphil']['dp']    *c['c1'] + h2o['dissolved']*phi['l']*c['dc1']['dp']  + h2o['dp']*phi['l']*c['c1']) 
    dMhldphig = m['rho_l']*( h2o['dissolved']*phi['dphil']['dphi_g']*c['c1'] ) 
    dMhldmh   = m['rho_l']*( h2o['dissolved']*phi['dphil']['dmh']   *c['c1'] + h2o['dissolved']*phi['l']*c['dc1']['dmh'] + h2o['dmh']*phi['l']*c['c1']) 
    
    # gas part
    dMhgdp    = 1/(1+gamma['g'])* var['phi_g']*rho['drhog']['dp']
    dMhgdphig = 1/(1+gamma['g'])*rho['g']
    dMhgdmh   = 1/(1+gamma['g'])*(var['phi_g']*rho['drhog']['dmh']) + rho['g']*var['phi_g']*gamma['dmh']
    
    
    J[:, m['vi']['phi_g'], m['vi']['vz']]     = fh.reshape(Nz)
    J[:, m['vi']['phi_g'], m['vi']['pz']]     = vrshp*(   dMhldp.reshape(Nz) +    dMhgdp.reshape(Nz)) #+ vgrshp*   dMhgdp.reshape(Nz) + fhg.reshape(Nz)*gvel['vg']['dp'].reshape(Nz)
    J[:, m['vi']['phi_g'], m['vi']['phi_gz']] = vrshp*(dMhldphig.reshape(Nz) + dMhgdphig.reshape(Nz)) #+ vgrshp*dMhgdphig.reshape(Nz) - fhg.reshape(Nz)*gvel['vg']['dphi_g'].reshape(Nz)
    J[:, m['vi']['phi_g'], m['vi']['mhz']]    = vrshp*(  dMhldmh.reshape(Nz) +   dMhgdmh.reshape(Nz)) #+ vgrshp*  dMhgdmh.reshape(Nz)
        
    Qh = 2.*rho['g']*var['phi_g']*gvel['ug']/m['R']/(1+gamma['g'])
    Q[:,m['vi']['phi_g']] = Qh.reshape(Nz)
    
    
    # carbon dioxide, separate into liquid and gas parts for slightly more intelligible code
    fcl = co2['dissolved']*m['rho_l']*phi['l']*c['c1']
    fcg = gamma['g']/(1+gamma['g'])*rho['g']*var['phi_g']
    fc  = fcl + fcg
    
    # liquid part
    dMcldp    = m['rho_l']*( co2['dissolved']*phi['dphil']['dp']    *c['c1'] + co2['dissolved']*phi['l']*c['dc1']['dp']  + co2['dp']*phi['l']*c['c1'] ) 
    dMcldphig = m['rho_l']*( co2['dissolved']*phi['dphil']['dphi_g']*c['c1'] ) 
    dMcldmh   = m['rho_l']*( co2['dissolved']*phi['dphil']['dmh']   *c['c1'] + co2['dissolved']*phi['l']*c['dc1']['dmh'] + co2['dmh']*phi['l']*c['c1']) 
    
    # gas part
    dMcgdp    = gamma['g']/(1+gamma['g'])* var['phi_g']*rho['drhog']['dp']
    dMcgdphig = gamma['g']/(1+gamma['g'])*rho['g']
    dMcgdmh   = gamma['g']/(1+gamma['g'])*(var['phi_g']*rho['drhog']['dmh']) - rho['g']*var['phi_g']*gamma['dmh']
    
    J[:, m['vi']['mh'], m['vi']['vz']]     = fc.reshape(Nz)
    J[:, m['vi']['mh'], m['vi']['pz']]     = vrshp*(   dMcldp.reshape(Nz) +    dMcgdp.reshape(Nz)) #+ vgrshp*   dMcgdp.reshape(Nz) + fcg.reshape(Nz)*gvel['vg']['dp'].reshape(Nz)
    J[:, m['vi']['mh'], m['vi']['phi_gz']] = vrshp*(dMcldphig.reshape(Nz) + dMcgdphig.reshape(Nz)) #+ vgrshp*dMcgdphig.reshape(Nz) - fcg.reshape(Nz)*gvel['vg']['dphi_g'].reshape(Nz)
    J[:, m['vi']['mh'], m['vi']['mhz']]    = vrshp*(  dMcldmh.reshape(Nz) +   dMcgdmh.reshape(Nz)) #+ vgrshp*  dMcgdmh.reshape(Nz)
    
    Qc = 2*gamma['g']*rho['g']*var['phi_g']*gvel['ug']/m['R']/(1+gamma['g'])
    Q[:,m['vi']['mh']] = Qc.reshape(Nz)
    
    return h2o, co2, c, phi, rho, fsl, fh, fc, J, Q
    

In [205]:
o = eng.tdcFV('setdef', nargout=1)
o['phi_gc'] = 0
ss, opts, ssflag = eng.tdcFV('run_ssc', o, nargout=3);
ss = eng.smf_rad_dz('add_fields',ss)

# get d mass matrix from symbolic manipulator results
dMsl, dMh, dMc = eng.pychecks(ss,nargout=3)
dMsl = mat_to_py(dMsl)
dMh = mat_to_py(dMh)
dMc = mat_to_py(dMc)


In [197]:
Nz    = np.int(opts['Nz'])
ind   = np.arange(0,Nz,100)
tmp   = -mat_to_py(ss['z'])
z     = tmp[ind]

p     = mat_to_py(ss['p'])
v     = mat_to_py(ss['v'])
phi_g = mat_to_py(ss['phi_g'])
mh    = mat_to_py(ss['mw'])

ddz   = mat_to_py(ss['yp'])

var          = {}
var['p']     = p[ind]
var['v']     = v[ind]
var['phi_g'] = phi_g[ind]
var['mh']    = mh[ind]

p = {}
v = {}
phi_g = {}
mh = {}

p['z']     = mat_to_py(ddz[ind,0])
v['z']     = mat_to_py(ddz[ind,1])
phi_g['z'] = mat_to_py(ddz[ind,2])
mh['z']    = mat_to_py(ddz[ind,3])

grd = {'p': p, 'v':v, 'phi_g':phi_g, 'mh':mh}
print(var['v'])

nu = np.squeeze(np.array([var['p'], var['v'],var['phi_g'],var['mh'],grd['p']['z'], grd['v']['z'] ,grd['phi_g']['z'],grd['mh']['z']] ).T)

# indices of variables
m['vi'] = {'p':0, 'v':1, 'phi_g':2, 'mh':3, 'pz':4, 'vz':5, 'phi_gz':6, 'mhz':7}



[[0.00091651]
 [0.00094601]
 [0.00098479]
 [0.00103332]
 [0.00108957]
 [0.0011804 ]
 [0.00114392]]


In [213]:


h2o, co2, c, phi, rho, fsl, fh, fc, J, Q = ssc(z, var, grd, m)

im = np.arange(0,301,50)

print('solids and liquids continuity')
print('matlab code: v x d sl/dp = ', var['v'].reshape(len(ind))*dMsl[im,0])
print('python code: v x d sl/dp = ', J[:,m['vi']['p'],m['vi']['pz']])
print('\n')

print('matlab code: v x d sl/dphig = ', var['v'].reshape(len(ind))*dMsl[im,2])
print('python code: v x d sl/dphig = ', J[:,m['vi']['p'],m['vi']['phi_gz']])
print('\n')

print('matlab code: v x d sl/dmh = ', var['v'].reshape(len(ind))*dMsl[im,3])
print('python code: v x d sl/dmh = ', J[:,m['vi']['p'],m['vi']['mhz']])
print('\n')

# NB If you want to compare the bottom two sets, you need to comment out the terms in ssc starting with vgrshp, 
# since the mass matrix does not include that
print('water continuity')
print('matlab code: v x d h/dp = ', var['v'].reshape(len(ind))*dMh[im,0])
print('python code: v x d h/dp = ', J[:,m['vi']['phi_g'],m['vi']['pz']])
print('\n')

print('matlab code: v x d h/dphig = ', var['v'].reshape(len(ind))*dMh[im,2])
print('python code: v x d h/dphig = ', J[:,m['vi']['phi_g'],m['vi']['phi_gz']])
print('\n')

print('matlab code: v x d h/dmh = ', var['v'].reshape(len(ind))*dMh[im,3])
print('python code: v x d h/dmh = ', J[:,m['vi']['phi_g'],m['vi']['mhz']])
print('\n')



print('co2 continuity')
print('matlab code: v x d c/dp = ', var['v'].reshape(len(ind))*dMc[im,0])
print('python code: v x d c/dp = ', J[:,m['vi']['mh'],m['vi']['pz']])
print('\n')

print('matlab code: v x d c/dphig = ', var['v'].reshape(len(ind))*dMc[im,2])
print('python code: v x d c/dphig = ', J[:,m['vi']['mh'],m['vi']['phi_gz']])
print('\n')

print('matlab code: v x d c/dmh = ', var['v'].reshape(len(ind))*dMc[im,3])
print('python code: v x d c/dmh = ', J[:,m['vi']['mh'],m['vi']['mhz']])
print('\n')




solids and liquids continuity
matlab code: v x d sl/dp =  [-8.49435274e-10 -8.30024223e-10 -9.13031736e-10 -1.13685590e-09
 -1.75665048e-09 -9.77238947e-09  0.00000000e+00]
python code: v x d sl/dp =  [-8.49435274e-10 -8.30024223e-10 -9.13031736e-10 -1.13685590e-09
 -1.75665048e-09 -9.77238947e-09 -7.03474613e-25]


matlab code: v x d sl/dphig =  [-2.12876053 -2.21003481 -2.31425562 -2.4440715  -2.5998315  -2.92868773
 -2.97420462]
python code: v x d sl/dphig =  [-2.12876053 -2.21003481 -2.31425562 -2.4440715  -2.5998315  -2.92868773
 -2.97420462]


matlab code: v x d sl/dmh =  [-2.76765577e-02 -2.38231616e-02 -2.02628188e-02 -1.68802250e-02
 -1.34067420e-02 -5.92610458e-03  5.42780683e-20]
python code: v x d sl/dmh =  [-2.76765577e-02 -2.38231616e-02 -2.02628188e-02 -1.68802250e-02
 -1.34067420e-02 -5.92610458e-03 -7.15603612e-20]


water continuity
matlab code: v x d h/dp =  [4.51062739e-10 5.12185838e-10 6.06436487e-10 7.37094643e-10
 9.25669926e-10 1.91687491e-09 7.25371607e-10]
py

In [203]:
#eng.quit()